(cont:mm:buffer-cache)=
# Buffer Cache
On a typical paged virtual memory system a very small number of page frames are actually in use at any one time.  All memory allocated to a program running in a process virtual address space is virtual memory.  When a program starts running none of the virtual pages map to physical page frames until the program references a virtual page without a mapping and a page fault occurs.  This happens repeatedly until the program maps an active subset of its virtual address space known as its process working set.  Over time all of the running programs create their own process working sets and collectively all processes create a system wide working set which is typically much smaller the all of the page frames on the entire system leaving a set of unused or free page frames.
Rather than allowing these free page frames to be unused the memory management system implements a Buffer Cache.  The Buffer Cache uses the free page frames as an in-memory cache of file system data and meta-data.

Each time a file system read occurs the Buffer Cache is searched to see if it can locate an in-memory copy of that data in one of the unused page frames.  If that data is found(AKA hit) it is simply copied from the page frame to the user buffer.  If that data is not found(AKA miss) the Buffer Cache allows the underlying file system to read the actual file data into a free page frame before copying it to the user buffer, afterwards any subsequent read for that data will be found in a free page frame(hit).  A file system write is similar to a read with the exception that the user buffer data is copied from the user buffer to one of the free page frames.  The underlying file system is called at some later time to write the contents of the associated free page frame to the file itself.  This concept of delaying the writing of modified data from the Buffer Cache to the actual file system is known as a "write-back cache".  This is in contrast with a "write-through cache" which would immediately write the modifications to the file system, blocking the write system call until its complete.

While it does add complexity to the memory management system the technique that the Buffer Cache uses of borrowing free page frames to cache file system data, copying between free page frames and user buffers and delaying the writes results in the file system read and write system calls being thousands or even millions times faster than doing actual file system IO operations.

